### Setting up the connection

In [ ]:
!pip install xgboost==2.0.1
!pip install catboost==1.2.2


In [ ]:
# Import Packages
import pymysql
import pandas as pd

In [ ]:
# Using mymysql to connect to ai300 remote database, these are the credentials
ENDPOINT = 'heicoders-playground.c2ced10ceyki.ap-southeast-1.rds.amazonaws.com'
PORT = 3306
USERNAME = 'student300'
DBNAME = 'ai300_capstone'
PASSWORD = 'heicoders_AI300'   # Default password is empty. Do change this if you had previously set a password.
CURSORCLASS = pymysql.cursors.DictCursor

In [ ]:
# This is the connection object
def initiate_local_connection():
    try:
        connection = pymysql.connect(host=ENDPOINT,
                                     port=PORT,
                                     user=USERNAME,
                                     passwd=PASSWORD,
                                     db=DBNAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Local Connection Successful')
    except Exception as e:
        print(f'[+] Local Connection Failed: {e}')
        connection = None

    return connection

In [ ]:
connection = initiate_local_connection()

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.account;'

def get_records(sql_query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)

        # Connection is not autocommit by default, so we must commit to save changes
        connection.commit()
        
        # Fetch all the records from SQL query output
        results = cursor.fetchall()
        
        # Convert results into pandas dataframe
        df = pd.DataFrame(results)
        
        print(f'Successfully retrieved records')
        
        return df
        
    except Exception as e:
        print(f'Error encountered: {e}')

### SQL queries to retrieve information from remote database

In [ ]:
sql_query = '''
        SELECT * 
        FROM churn_status cs
        INNER JOIN customer c ON cs.customer_id = c.customer_id 
        INNER JOIN account a ON cs.customer_id = a.customer_id
        INNER JOIN city ci ON c.zip_code = ci.zip_code
        INNER JOIN account_usage au ON a.account_id = au.account_id;
'''

combined_df = get_records(sql_query)
combined_df

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.account;'

df1 = get_records(sql_query)
df1

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.account_usage;'

df2 = get_records(sql_query)
df2

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.churn_status;'

df3 = get_records(sql_query)
df3

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.city;'

df4 = get_records(sql_query)
df4

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.customer;'

df5 = get_records(sql_query)
df5

### Dropping all repeated columns from the combined dataframe

In [ ]:
combined_df = combined_df.drop(
    columns=['c.customer_id','a.customer_id','ci.zip_code','au.account_id']
)

combined_df

### EDA 
#### Currently there are df1 to df5, each representing an extracted table, while combined_df is all data in 1 table

In [ ]:
# since all tables have been combined together
pd.options.display.max_columns = 50

In [ ]:
combined_df.head()

# target parameter is churn label

In [ ]:
combined_df.info()

In [ ]:
combined_df.nunique()

In [ ]:
df1.info()
print()
df2.info()
print()
df3.info()
print()
df4.info()
print()
df5.info()
print()

In [ ]:
df1.nunique()

In [ ]:
df5.nunique()

In [ ]:
import plotly.express as px

corr = combined_df.corr()
fig = px.imshow(corr, color_continuous_scale = 'Brwnyl', text_auto = True)
fig.update_xaxes(side="top")
fig.show()


### Preliminary Visualizations

In [ ]:
import matplotlib.pyplot as plt 
import plotly.express as px

In [ ]:
fig = px.scatter_matrix(combined_df, dimensions=['zip_code', 'city', 'latitutde','longitude','population'],
                       color='churn_label')


fig.show()


In [ ]:
fig = px.scatter_matrix(combined_df, dimensions=['gender', 'age', 'senior_citizen','married','num_dependents'],
                       color='churn_label')


fig.show()


In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['avg_long_distance_fee_monthly', 'total_long_distance_fee', 
                                    'avg_gb_download_monthly','stream_tv','stream_movie','stream_music'
                                   ,'total_monthly_fee','total_charges_quarter','total_refunds'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['avg_long_distance_fee_monthly', 'total_long_distance_fee',
                                    'total_charges_quarter','avg_gb_download_monthly' 
                                  ],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['tenure_months', 'num_referrals', 
                                    'has_internet_service','internet_type','has_unlimited_data','has_phone_service',
                                   'has_multiple_lines','has_premium_tech_support','has_online_security',
                                   'has_online_backup','has_device_protection','contract_type','paperless_billing',
                                   'payment_method'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['tenure_months', 'num_referrals', 
                                    'internet_type','contract_type',
                                   'payment_method'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='num_referrals',
                    height = 400)


fig.show()

#num_referrals 0 has a larger portion

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='internet_type',
                    height = 400)


fig.show()

#internet type fiber optic is a bigger portion

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='contract_type',
                    height = 400)


fig.show()

#contract_type 0 has a larger portion of leavers

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='population',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='tenure_months',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_internet_service',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='internet_type',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_unlimited_data',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_phone_service',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_multiple_lines',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_premium_tech_support',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_online_security',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_online_backup',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_device_protection',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='paperless_billing',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='payment_method',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_tv',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_movie',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_music',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='gender',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='age',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='senior_citizen',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='married',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='num_dependents',
                    height = 400)


fig.show()


In [ ]:
fig = px.bar(combined_df['payment_method'].value_counts()) 


fig.show()

In [ ]:
fig = px.bar(combined_df['churn_label'].value_counts()) 
            
fig.show()
# From dataset, how many customers have churned in the last month

In [ ]:
yes_churn_df = combined_df[combined_df.churn_label == "Yes"]

fig = px.bar(yes_churn_df['churn_category'].value_counts()) 
            
fig.show()
# From those that have churned, what is the main cause? Competitor

### Based on the above, features chosen  are 
### has_premium_tech_support, contract_type, internet_type, has_unlimited_data, num_dependents, num_referals

### Quick EDA of the new DF to decide on how to engineer the features

In [ ]:
combined_df['num_dependents']

In [ ]:
combined_df['churn_label'].unique()

In [ ]:
selected_df = combined_df[['has_premium_tech_support','contract_type','internet_type',
             'has_unlimited_data','num_dependents', 'num_referrals',
             'churn_label']]

selected_df

In [ ]:
selected_df.nunique()

In [ ]:
selected_df['has_unlimited_data'].unique()

# to check out the values within the values all columns

In [ ]:
selected_df.info()

# no missing data found, most of the columns are categorical data

### Feature Engineering

In [ ]:

# replacing categorical data denoted by strings in each column to numerical values
selected_df['has_premium_tech_support'].replace(['Yes','No'],[1,0], inplace=True) 
selected_df['contract_type'].replace(['Month-to-Month', 'Two Year', 'One Year'],[0,1,2], inplace=True) 
selected_df['internet_type'].replace(['Fiber Optic', 'Cable', 'DSL', 'None'],[1,2,3,0], inplace=True) 
selected_df['has_unlimited_data'].replace(['Yes','No'],[1,0], inplace=True)

selected_df['churn_label'].replace(['Yes','No',''],[1,0,0], inplace=True)
selected_df




### Testing out xgb as 1st model

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = xgb.XGBClassifier(learning_rate=0.2, max_depth=2, min_child_weight=5, random_state=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

In [ ]:
from catboost import CatBoostClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = CatBoostClassifier(learning_rate=0.1, depth=1, random_seed=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# this was performed for xgBoost
param_grid = {
    'max_depth': [1,2,3,4,5],
    'min_child_weight': [1,2,3,4,5],
    'eta': [0.1, 0.3, 0.5]
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=param_grid, cv=10)
grid_search.fit(X_train, y_train)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'depth': [1,2,3,4,5],
    'learning_rate': [0.1]
}

grid_search = GridSearchCV(estimator= CatBoostClassifier(), param_grid=param_grid, cv=10)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator
best_estimator = grid_search.best_estimator_

# Get the best hyperparameters
best_params = grid_search.best_params_

print(best_estimator)
print()
print(best_params)

In [ ]:
from catboost import CatBoostClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = CatBoostClassifier(learning_rate=0.1, depth=2, random_seed=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

Exporting Model

In [ ]:
from pathlib import Path

Path("../model").mkdir(exist_ok=True)

In [ ]:
import joblib

joblib.dump(model, '../model/catboost_model.pkl')